In [9]:
from h5_funcs import *
import h5py


In [2]:
user = 'maansidesai'
#subject = 'MT0031'
subject_list = ['MT0028', 'MT0029', 'MT0030','MT0031' ,'MT0032', 'MT0033', 'MT0034', 'MT0035', 'MT0036', 'MT0037', 'MT0038']

datadir = f'/Users/{user}/Box/MovieTrailersTask/Data/EEG/Participants/'
#event_file = f'{datadir}/{subject}/audio/{subject}_B{block}_MovieTrailers_events.txt'
trailer_file = f'{datadir}/event_files/trailer_phn_info_index.txt'

wav_dir = f'/Users/{user}/Box/Stimuli/MovieTrailers/'
fs=128.
frame_rate_sec = 0.04170833333333333



In [3]:
#change this based on block number and the stimulus type (AV, V-only, A-only)
block=1
condition = ['AV', 'A', 'V']
#condition = ['V']

In [4]:
# Now create large .h5 file and append all blocks and Movie Trailer data 
for subject in subject_list:
	with h5py.File(f'%s/full_AV_matrix.hf5'%(datadir), 'a') as g:
		
		epochs_list = dict()
		for idx, s in enumerate(condition):
			epochs_list[s] = dict()
			print(s)
			print(idx)
			event_file = pd.read_csv(f'{datadir}/{subject}/audio/{subject}_B{block}_MovieTrailers_events.csv')

			a = np.where(event_file['condition'] == s)
			print(a)
			evs = event_file.iloc[a][['onset_time', 'offset_time', 'event_id']].values
			evs[:,:2] = evs[:,:2]*fs
			print(evs.astype(np.int))
			if s == 'V':
				print('Visual condition')
				evs[:,:2] = evs[:,:2] - (frame_rate_sec *fs) #subtract frame rate
			else:
				print('NOT PROCESSING Visual condition')
			evs = evs.astype(np.int)
			print(evs)
			wav_id = event_file.iloc[a]['name'].values 
			wav_id = wav_id + '.wav'
			unique_stimuli = np.unique(evs[:,2])
			unique_ids = np.unique(wav_id)

			raw = load_raw_EEG(subject, block, datadir)
			new_fs = np.int(raw.info['sfreq'])
			for indx, event in enumerate(evs):
				print('*******************')
				print('Printing event: ')
				#print(event)

				wav_name = wav_id[indx]
				print(wav_name)
				event_id = event[2]

				epochs = get_event_epoch(raw, evs, event_id)

				print(event_id)

				binary_feat_mat, binary_phn_mat = binary_phn_mat_stim(datadir, trailer_file, subject, block, wav_name, epochs, s, fs=128.)

				print(binary_phn_mat.shape)
				try:
					g.create_dataset('%s/%s/resp/%s/epochs'%(s, wav_name, subject), data=np.array(epochs, dtype=float))
				except:
					print('Epochs already exist for trailer')
				try:
					g.create_dataset('%s/%s/stim/phn_feat_timings'%(s, wav_name), data=np.array(binary_feat_mat, dtype=float))
					g.create_dataset('%s/%s/stim/phn_timings'%(s, wav_name), data=np.array(binary_phn_mat, dtype=float))

				except:
					print('phn_timings already exists')

				# Now add stimulus representations
				if s == 'V':
					print('processing visual only stimuli...not adding auditory feats')
					continue 
				

				print('*********************************************')
				print(wav_name)
				print('*********************************************')

				if '%s/%s/stim/spec'%(s, wav_name) not in g:

					envelope = make_envelopes(wav_dir, wav_name, new_fs, epochs, pad_next_pow2=True)

					mel_spec, freqs = stimuli_mel_spec(wav_dir, wav_name)
					pitch_values = get_meanF0s_v2(fileName=wav_dir+wav_name)


					#g.create_dataset('/%s/%s/stim/phn_time' %(s, wav_name), data=np.array(phoneme_times, dtype=float))  

					g.create_dataset('%s/%s/stim/spec'%(s, wav_name), data=np.array(mel_spec, dtype=float))
					g.create_dataset('%s/%s/stim/freqs'%(s, wav_name), data=np.array(freqs, dtype=float))
					g.create_dataset('%s/%s/stim/pitches'%(s, wav_name), data=np.array(pitch_values, dtype=float))
					g.create_dataset('%s/%s/stim/envelope'%(s, wav_name), data=np.array(envelope, dtype=float))

					print('all stims already exists')

AV
0
(array([ 0,  2,  3,  4,  5, 12, 18, 21]),)
[[  1234  19302      0]
 [ 37430  57435      1]
 [ 57635  76787      2]
 [ 76939  94302      3]
 [ 94436 114436      6]
 [231061 248487      5]
 [339530 355199      7]
 [392881 411882      4]]
NOT PROCESSING Visual condition
[[  1234  19302      0]
 [ 37430  57435      1]
 [ 57635  76787      2]
 [ 76939  94302      3]
 [ 94436 114436      6]
 [231061 248487      5]
 [339530 355199      7]
 [392881 411882      4]]
Opening raw data file /Users/maansidesai/Box/MovieTrailersTask/Data/EEG/Participants//MT0028/downsampled_128/MT0028_B1_postICA_rejected.fif...
    Range : 0 ... 448690 =      0.000 ...  3505.391 secs
Ready.


<ipython-input-4-4682a863b75f>:16: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(evs.astype(np.int))
<ipython-input-4-4682a863b75f>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more detai

Reading 0 ... 448690  =      0.000 ...  3505.391 secs...
Bad channels are: 
[]
(64, 448691)
*******************
Printing event: 
pele-tlr1_a720p.wav
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18069 original time points ...
0 bad epochs dropped
0
18069
(59, 18069)


<ipython-input-4-4682a863b75f>:30: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  new_fs = np.int(raw.info['sfreq'])


sonorant
obstruent
voiced
back
front
low
high
dorsal
coronal
labial
syllabic
plosive
fricative
nasal
(59, 18069)
*********************************************
pele-tlr1_a720p.wav
*********************************************
Sentence: pele-tlr1_a720p.wav
Padding the signal to the nearest power of two...this should speed things up
calculating hilbert transform
Low-pass filtering hilbert transform to get audio envelope
Removing padding
(33554, 1)
2258466
(18067, 1)


...

sonorant
obstruent
voiced
back
front
low
high
dorsal
coronal
labial
syllabic
plosive
fricative
nasal
(59, 20007)
phn_timings already exists
*********************************************
bighero6-tlr3_a720p.wav
*********************************************
*******************
Printing event: 
thelittleprince-tlr_a720p.wav
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
0 bad epochs dropped
6
20001
(59, 20001)
sonorant
obstruent
voiced
back
front
low
high
dorsal
coronal
labial
syllabic
plosive
fricative
nasal
(59, 20001)
phn_timings already exists
*********************************************
thelittleprince-tlr_a720p.wav
*********************************************
*******************
Printing event: 
pele-tlr1_a720p.wav
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Load

<ipython-input-4-4682a863b75f>:16: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(evs.astype(np.int))
<ipython-input-4-4682a863b75f>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more detai

Bad channels are: 
[]
(64, 444230)
*******************
Printing event: 
pele-tlr1_a720p_visualonly_notif.wav
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18069 original time points ...
0 bad epochs dropped
0
18069
(59, 18069)


<ipython-input-4-4682a863b75f>:30: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  new_fs = np.int(raw.info['sfreq'])


sonorant
obstruent
voiced
back
front
low
high
dorsal
coronal
labial
syllabic
plosive
fricative
nasal
(59, 18069)
phn_timings already exists
processing visual only stimuli...not adding auditory feats
*******************
Printing event: 
insideout-tlr2zzyy32_a720p_visualonly_notif.wav
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 17364 original time points ...
0 bad epochs dropped
3
17364
(59, 17364)
sonorant
obstruent
voiced
back
front
low
high
dorsal
coronal
labial
syllabic
plosive
fricative
nasal
(59, 17364)
phn_timings already exists
processing visual only stimuli...not adding auditory feats
*******************
Printing event: 
deep-trailer-1_a720p_visualonly_notif.wav
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 15670 original time points ...
0 bad epochs dropped
7


# Append scene cut information to V-only and AV .h5 files - ONLY DO THIS ONCE

In [11]:
h5_dir = datadir
textgrid_dir = '/Users/maansidesai/Box/trailer_AV/textgrids/scene_cut_textGrids/AV_task'
#subject = 'MT0021'
filename = 'full_AV_matrix'
condition = ['V']
for i in condition:
    scene_cut_feature(h5_dir, textgrid_dir, filename, i, fs=128.0)


/Users/maansidesai/Box/trailer_AV/textgrids/scene_cut_textGrids/AV_task/bighero6-tlr3_a720p_corrected_SC.TextGrid
bighero6-tlr3_a720p.wav
(20005,)
/Users/maansidesai/Box/trailer_AV/textgrids/scene_cut_textGrids/AV_task/deep-trailer-1_a720p_corrected_SC.TextGrid
deep-trailer-1_a720p.wav
(15668,)
/Users/maansidesai/Box/trailer_AV/textgrids/scene_cut_textGrids/AV_task/ferdinand-trailer-2_a720p_corrected_SC.TextGrid
ferdinand-trailer-2_a720p.wav
(19152,)
/Users/maansidesai/Box/trailer_AV/textgrids/scene_cut_textGrids/AV_task/insideout-tlr2zzyy32_a720p_corrected_SC.TextGrid
insideout-tlr2zzyy32_a720p.wav
(17362,)
/Users/maansidesai/Box/trailer_AV/textgrids/scene_cut_textGrids/AV_task/insideout-usca-tlr2_a720p_corrected_SC.TextGrid
insideout-usca-tlr2_a720p.wav
(19000,)
/Users/maansidesai/Box/trailer_AV/textgrids/scene_cut_textGrids/AV_task/paddington-2-trailer-1_a720p_corrected_SC.TextGrid
paddington-2-trailer-1_a720p.wav
(17424,)
/Users/maansidesai/Box/trailer_AV/textgrids/scene_cut_textGr

# Copy gabor_pc10 from older EEG (full matrix from JNeuro 2021 paper) to the visual and audiovisual .h5 files

Only do this once

In [8]:
mts = ['bighero6-tlr3_a720p.wav',
 'deep-trailer-1_a720p.wav',
 'ferdinand-trailer-2_a720p.wav',
 'insideout-tlr2zzyy32_a720p.wav',
 'insideout-usca-tlr2_a720p.wav',
 'paddington-2-trailer-1_a720p.wav',
 'pele-tlr1_a720p.wav',
 'thelittleprince-tlr_a720p.wav']
condition = 'A'


with h5py.File(f'{datadir}/full_AV_matrix.hf5','a') as f_dest: #file to copy gabors to
    with h5py.File('/Users/maansidesai/Desktop/data/fullEEGmatrix.hf5','r') as f_src: #full EEG matrix
        for idx, i in enumerate(mts):
            gabor = f_src[f'MovieTrailers/{i}/stim/gabor_pc10'][:]
            print(i)
            print(gabor.shape)
            if condition == 'V':
                name = i.replace('.wav', '_visualonly_notif.wav')
                print(name)
            else:
                name = i
            f_dest.create_dataset(f'{condition}/{name}/stim/gabor_pc10', data=np.array(gabor, dtype=float))

bighero6-tlr3_a720p.wav
(20006, 10)
deep-trailer-1_a720p.wav
(15670, 10)
ferdinand-trailer-2_a720p.wav
(19153, 10)
insideout-tlr2zzyy32_a720p.wav
(17364, 10)
insideout-usca-tlr2_a720p.wav
(19002, 10)
paddington-2-trailer-1_a720p.wav
(17427, 10)
pele-tlr1_a720p.wav
(18068, 10)
thelittleprince-tlr_a720p.wav
(20001, 10)


# Append congruent and incongruent mouthing information

In [ ]:
textgrid_talker_dir = f'{textgrid_dir}/talker_tg'
subject = 'MT0028' #pick random subject because you're just getting a binary matrix of 0s and 1s for talking info per trailer

face_mat =  face_talker(datadir, textgrid_talker_dir, subject, 'AV') #run for audiovisual condition
face_mat =  face_talker(datadir, textgrid_talker_dir, subject, 'V') #run for visual only condition
